- I could be mistaken, but maybe there is an error in the TFRecord file.
- Images in jpeg and tfrecord files may be different.

In [ ]:
import os
import cv2
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf


DATA_PATH = '/kaggle/input/cassava-leaf-disease-classification'

JPEG_PATH = os.path.join(DATA_PATH, 'train_images')
CSV_PATH = os.path.join(DATA_PATH, 'train.csv')

files = sorted(os.listdir(JPEG_PATH))
train_df = pd.read_csv(CSV_PATH)
targets = train_df['label']

train_df.head(25)

## JPEG check

In [ ]:
def jpeg_display(directory_path):
    fig, axes = plt.subplots(5, 5, figsize=(16, 16))
    for i in range(25):
        img = cv2.imread(os.path.join(directory_path, files[i]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        axes[i//5][i%5].imshow(img)
        axes[i//5][i%5].set_title(f'{files[i]}: {targets[i]}')
    plt.show()
    
    
jpeg_display(JPEG_PATH)

## TFRecords check

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def parse_example(example):
    LABELED_TFREC_FORMAT = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.int64)
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = example['image_name']
    target = example['target']
    return image, label, target


def display_one(image, title, target, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(f'{title}: {target}')
    return (subplot[0], subplot[1], subplot[2]+1)


def display_batch_of_images(databatch):
    images, labels, targets = databatch
    images = images.numpy()
    labels = labels.numpy()
    targets = targets.numpy()
    if labels is None:
        labels = [None for _ in enumerate(images)]
    if targets is None:
        targets = [None for _ in enumerate(targets)]
        
    # auto-squaring: this will drop data that does not fit into square or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.2
    subplot=(rows, cols, 1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label, target) in enumerate(zip(images[:rows*cols], labels[:rows*cols], targets[:rows*cols])):
        title = label
        title = title.decode('utf-8')
        correct = True
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one(image, title, target, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0.2, hspace=0.2)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

In [ ]:
file = '/kaggle/input/cassava-leaf-disease-classification/train_tfrecords/ld_train00-1338.tfrec'
dataset = tf.data.TFRecordDataset([file]).map(parse_example).batch(25)
data = iter(dataset)
display_batch_of_images(next(data))